<a href="https://colab.research.google.com/github/notanumber-sys/dd2365-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Surface Waves in Steady State Shallow Flow by Navier-Stokes Equations**
**Ludwig Karlsson**, based on template by **Johan Hoffman**.

# **Set up environment**

In [94]:
# Load neccessary modules.
from google.colab import files

import numpy as np
import time

# Install FEniCS (this may take a long time)
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
    
from dolfin import *; from mshr import *

import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt

# **Method**

**Launch Options**

Change the value of the launch configurator option to choose which experiment to run.

In [95]:
# Experiment
# 0: Closed tub experiment
# 1: Type 1 flow
# 2: Type 2 flow
# 3: Type 3 flow
# 4: Type 4 flow
# 5: Type 3 flow alt.

launch_configuration = 0

# Define rectangular domain 
L = 10
H = 1

# bottom shape parameters
B0 = H/10
r = L/6

# initial wave shape
wave_epsilon = H/2
wave_width = L/10
apply_initial_surface_deformation = False

# mesh resolution
resolution = 12

# inflow velocity (for open experiments)
U = 1.2

# choose boundary conditions
closed = False

# for outflow, the goal is to be able to expell excess volume if necessary
use_velocity_limits = False

if launch_configuration == 0:   # closed tub experiment
    closed = True
    time_step_size_quotient = 20.0
    T = 5.0

    # wave:
    wave_shape = Expression("H + weps*exp(-pow(x[0] - L/2, 2)/(ww*ww))", H=H, weps=wave_epsilon, L=L, ww=wave_width, degree=2)
    wave_shape_displacement = Expression(("0", "(weps*exp(-pow(x[0] - L/2, 2)/(ww*ww)))*(x[1] > (H - EH))*((x[1] - H + EH)/EH)"), H=H, weps=wave_epsilon, L=L, ww=wave_width, EH=effective_height, degree=2)
    apply_initial_surface_deformation = True
elif launch_configuration == 1: # type 1 flow
    time_step_size_quotient = 20.0
    T = 5.0

    # flat:
    wave_shape = Expression("H", H=H, degree=2)
elif launch_configuration == 2: # type 2 flow
    U = 4.8
    time_step_size_quotient = 40.0
    T = 2.9

    # flat:
    wave_shape = Expression("H", H=H, degree=2)
elif launch_configuration == 3: # type 3 flow
    B0 = H/3
    time_step_size_quotient = 30.0
    T = 0.55
    
    use_velocity_limits = True
    max_velocity = 1.2*U
    min_velocity = 0.8*U

    # inverted wave:
    wave_shape = Expression("H - weps*exp(-pow(x[0] - L/2 - 1, 2)/(ww*ww))", H=H, weps=H/3, L=L, ww=wave_width, degree=2)
    wave_shape_displacement = Expression(("0", "-(weps*exp(-pow(x[0] - L/2 - 1, 2)/(ww*ww)))*(x[1] > (H - EH))*((x[1] - H + EH)/EH)"), H=H, weps=H/3, L=L, ww=wave_width, EH=effective_height, degree=2)
    apply_initial_surface_deformation = True
elif launch_configuration == 4: # type 4 flow
    B0 = H/5
    U = 3.0
    time_step_size_quotient = U*10
    T = 2.9
    
    use_velocity_limits = True
    max_velocity = 2*U
    min_velocity = 1.5*U

    # negative sigmoid:
    wave_shape = Expression("H + 1/(1 + exp(2*(x[0] - 5)))", H=H, degree=2)
    wave_shape_displacement = Expression(("0", "(1/(1 + exp(2*(x[0] - 5))))*(x[1] > (H - EH))*((x[1] - H + EH)/EH)"), H=H, weps=wave_epsilon, L=L, ww=wave_width, EH=effective_height, degree=2)
    apply_initial_surface_deformation = True
elif launch_configuration == 5: # potentially type 3
    U = 2.4
    time_step_size_quotient = U*10
    T = 3.0

    # flat
    wave_shape = Expression("H", H=H, degree=2)

effective_height = H - B0



**Define domain and mesh**

In [ ]:

# Define bump
bottom_shape = Expression("B0*pow(cos(pi*(x[0] - L/2)/(2*r)), 2.0)*(abs(x[0] - L/2) < r)", B0=B0, L=L, r=r, degree=2)
shape_displacement = Expression(("0", "(B0*pow(cos(pi*(x[0] - L/2)/(2*r)), 2.0)*(abs(x[0] - L/2) < r))*((H - x[1])/H)"), B0=B0, L=L, r=r, H=H, degree=2)

# Define subdomains (for boundary conditions)
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0.0) 

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], L)

class Lower(SubDomain):
    def inside(self, x, on_boundary):
        if on_boundary and (not near(x[0], 0.0)) and (not near(x[0], L)) and x[1] <= 1.1*B0:
            return True
        if near(x[1], 0.0) and (near(x[0], 0.0) or near(x[0], L)):
            return True
        return False

class Upper(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], wave_shape(x))
      
left = Left()
right = Right()
lower = Lower()
upper = Upper()

# Generate mesh
delta_x = 1/resolution
mesh = RectangleMesh(Point(0.0, 0.0), Point(L, H), L*resolution, H*resolution)

ALE.move(mesh, shape_displacement)
# displace for initial wave shape
if apply_initial_surface_deformation:
    ALE.move(mesh, wave_shape_displacement)

# Define mesh functions (for boundary conditions)
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
boundaries.set_all(0)
left.mark(boundaries, 1)
right.mark(boundaries, 2)
lower.mark(boundaries, 3)
upper.mark(boundaries, 4)

plt.figure(figsize=(40, 4))
plot(mesh, title="Initial Mesh")
plt.show()

**Define finite element approximation spaces**

In [97]:
# Generate finite element spaces (for velocity and pressure)
V = VectorFunctionSpace(mesh, "Lagrange", 1)
Q = FunctionSpace(mesh, "Lagrange", 1)

# Define trial and test functions 
u = TrialFunction(V)
p = TrialFunction(Q)
v = TestFunction(V)
q = TestFunction(Q)

**Define boundary conditions**

In [98]:
# Define boundary conditions 
class DirichletBoundaryLower(SubDomain):
    def inside(self, x, on_boundary):
        if on_boundary and (not near(x[0], 0.0)) and (not near(x[0], L)) and x[1] <= 1.1*B0:
            return True
        if near(x[1], 0.0) and (near(x[0], 0.0) or near(x[0], L)):
            return True
        return False

class DirichletBoundaryUpper(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (not near(x[0], 0.0)) and (not near(x[0], L)) and x[1] > 1.1*B0

class DirichletBoundaryLeft(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], 0.0) 

class DirichletBoundaryRight(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], L)

dbc_lower = DirichletBoundaryLower()
dbc_upper = DirichletBoundaryUpper()
dbc_left = DirichletBoundaryLeft()
dbc_right = DirichletBoundaryRight()

# in flow
Vdot = wave_shape(0.0)*U

class InVelocityExpression(UserExpression):
    def eval(self, value, x):
        K = Vdot/self.h
        value[0] = K

    def value_shape(self):
        return ()
uin = InVelocityExpression()
uin.h = wave_shape(0.0)

class OutVelocityExpression(UserExpression):
    def eval(self, value, x):
        K = Vdot/self.h

        if use_velocity_limits:
            if K < min_velocity:
                K = min_velocity
            elif K > max_velocity:
                K = max_velocity
        value[0] = K

    def value_shape(self):
        return ()
uout = OutVelocityExpression()
uout.h = H

if closed:
    bcu_left0 = DirichletBC(V.sub(0), 0.0, dbc_left)
    bcu_right0 = DirichletBC(V.sub(0), 0.0, dbc_right)
    bcu_low0 = DirichletBC(V.sub(0), 0.0, dbc_lower)
    bcu_low1 = DirichletBC(V.sub(1), 0.0, dbc_lower)

    bcp0 = DirichletBC(Q, 0.0, dbc_upper)
    
    bcu = [bcu_left0, bcu_right0, bcu_low0, bcu_low1]
    bcp = [bcp0]
else:
    bcu_left0 = DirichletBC(V.sub(0), uin, dbc_left)
    bcu_right0 = DirichletBC(V.sub(0), uout, dbc_right)
    bcu_low0 = DirichletBC(V.sub(0), 0.0, dbc_lower)
    bcu_low1 = DirichletBC(V.sub(1), 0.0, dbc_lower)

    bcp0 = DirichletBC(Q, 0.0, dbc_upper)

    bcu = [bcu_left0, bcu_right0, bcu_low0, bcu_low1]
    bcp = [bcp0]


# **Results**

**Define flow parameters**

In [99]:
# Set viscosity (water)
nu = 1.0e-6 # m²/s
rho = 1e3   # kg/m³

# set gravity
g = 9.81    # m/s²
f = Expression(("0", "-g"), g=g, degree=2);

**Initial Conditions**

In [100]:
class InitialPressureExpression(UserExpression):
  def eval(self, value, x):
    value[0] = (wave_shape(x) - x[1])*g*rho

  def value_shape(self):
    return ()

class InitialVelocityExpression(UserExpression):
  def eval(self, value, x):
    value[0] = uin(x[0], x[1])*(wave_shape(0.0))/(wave_shape(x[0]) - bottom_shape(x[0]))
    value[1] = 0.0

  def value_shape(self):
    return (2,)

u_init = InitialVelocityExpression()
p_init = InitialPressureExpression()

**Define method parameters**

In [101]:
# Define iteration functions
# (u0,p0) solution from previous time step
# (u1,p1) linearized solution at present time step  
u0 = Function(V)
u1 = Function(V)
p0 = Function(Q)
p1 = Function(Q)

class DisplacementExpression(UserExpression):
    def set_md(self, displacement):
        if displacement == None:
            self.mesh_displacement = None
        else:
            self.mesh_displacement = interpolate(displacement, V)
            self.mesh_displacement.set_allow_extrapolation(True)
    
    def eval(self, value, x):
        if self.mesh_displacement is None:
            value[0] = 0.0
            value[1] = 0.0
        else:
            result = self.mesh_displacement(x[0], x[1])
            value[0] = result[0]
            value[1] = result[1]

    def value_shape(self):
        return (2,)
w = DisplacementExpression()
w.set_md(None)

# Set parameters for nonlinear and lienar solvers 
num_nnlin_iter = 5 
prec = "amg" if has_krylov_solver_preconditioner("amg") else "default" 

# Time step length
dt = 0.5*mesh.hmin() /time_step_size_quotient

**Define variational problem**

In [102]:
# Define variational problem

# Stabilization parameters
h = CellDiameter(mesh);
u_mag = sqrt(dot(u1,u1))
d1 = 1.0/sqrt((pow(1.0/dt,2.0) + pow(u_mag/h,2.0)))
d2 = h*u_mag

# Mean velocities for trapozoidal time stepping
um = 0.5*(u + u0)
um1 = 0.5*(u1 + u0)

# Momentum variational equation on residual form
Fu = inner((u - u0)/dt + grad(um)*(um1-w/dt), v)*dx \
    - p1*div(v)*dx \
    + nu*inner(grad(um), grad(v))*dx \
    + d1*inner((u - u0)/dt + grad(um)*(um1-w/dt) + (1/rho)*grad(p1), grad(v)*(um1-w/dt))*dx \
    + d2*div(um)*div(v)*dx \
    - inner(f, v)*dx
au = lhs(Fu)
Lu = rhs(Fu)

# Continuity variational equation on residual form
Fp = d1*inner((u1 - u0)/dt + grad(um1)*(um1-w/dt) + grad(p), grad(q))*dx + div(um1)*q*dx 
ap = lhs(Fp)
Lp = rhs(Fp)

**Set plotting variables and open export files**

In [103]:
# Open files to export solution to Paraview
file_u = File("results-NS/u.pvd")
file_p = File("results-NS/p.pvd")

# Set plot frequency
plot_time = 0
plot_freq = 20

# Time data 
time = np.array(0.0)
time = np.delete(time, 0)
start_sample_time = 1.0

**Time stepping algorithm**

In [ ]:
!rm results-NS/*

# apply initial conditions
if not closed:
    u0.interpolate(u_init)
    u1.interpolate(u_init)

print("Initial Pressure")
plt.figure(figsize=(40, 4))
plot(p1, title="Pressure")
print("===========")

print("Initial Velocity")
plt.figure(figsize=(40, 4))
plot(u1, title="Velocity")
print("===========")


# Time stepping
t = dt

def surface_node_compare(surface_node):
    return surface_node.position[0]

# used for boundary deformation scheme
class SurfaceNode():
    def __init__(self, mesh_index, position, u):
        self.mesh_index = mesh_index
        self.position = position
        self.velocity = u(position[0], position[1])

    def setup(self, surface):
        if self.index == 0:
            self.k = (self.position[1] - surface[1].position[1])/delta_x
        elif self.index == len(surface) - 1:
            self.k = (surface[-2].position[1] - self.position[1])/delta_x
        else:
            self.k = (surface[self.index + 1].position[1] - surface[self.index - 1].position[1])/(2*delta_x)

        self.k_scale = 1/sqrt(1 + self.k**2)
        self.delta_s = delta_x/self.k_scale
        self.n = [-self.k*self.k_scale, self.k_scale]
        self.Vdot = (self.n[0]*self.velocity[0] + self.n[1]*self.velocity[1])*self.delta_s*dt
        self.Vxdot = self.n[0]*self.Vdot
        self.Vydot = self.n[1]*self.Vdot
        self.W = self.Vydot/delta_x
        self.Wp = max(0, self.Vxdot/delta_x)
        self.Wm = max(0, -self.Vxdot/delta_x)

    def timestep(self, surface, boundary_coordinates):
        x = boundary_coordinates[self.mesh_index]
        x[1] += self.W
        if self.index != 0:
            x[1] += surface[self.index - 1].Wp
        elif closed:
            x[1] += self.Wm
        if self.index != len(surface) - 1:
            x[1] += surface[self.index + 1].Wm
        elif closed:
            x[1] += self.Wp
        else: # this builds on increasing update order
            x[1] = surface[self.index - 1].position[1]
        self.position = x

while t < T + DOLFIN_EPS:
    # extract the boundary mesh for the surface
    boundary = BoundaryMesh(mesh, "exterior", True)

    surface_nodes = []
    left_node_indices = []   # not including top
    right_node_indices = []  # not including top
    left_max = 0
    left_max_index = -1
    right_max = 0
    right_max_index = -1
    for i, x in enumerate(boundary.coordinates()):
        if lower.inside(x, True):
            pass
        elif left.inside(x, True):
            left_node_indices.append(i)
            if x[1] > left_max:
                left_max = x[1]
                left_max_index = i
        elif right.inside(x, True):
            right_node_indices.append(i)
            if x[1] > right_max:
                right_max = x[1]
                right_max_index = i
        else:
            surface_nodes.append(SurfaceNode(i, x, u1))
    surface_nodes.insert(0, SurfaceNode(left_max_index, [0.0, left_max], u1))
    surface_nodes.append(SurfaceNode(right_max_index, [L, right_max], u1))
    left_node_indices.remove(left_max_index)
    right_node_indices.remove(right_max_index)

    surface_nodes.sort(key=surface_node_compare)

    # displace surface boundary
    for i, node in enumerate(surface_nodes):
        node.index = i
    for node in surface_nodes:
        node.setup(surface_nodes)
    for node in surface_nodes:
        node.timestep(surface_nodes, boundary.coordinates())

    # deform edge boundary
    new_y_left = boundary.coordinates()[left_max_index][1]
    for j, i in enumerate(left_node_indices):
        x = boundary.coordinates()[i]
        positional_scale = x[1]/left_max
        x[1] = new_y_left*positional_scale
    new_y_right = boundary.coordinates()[right_max_index][1]
    for j, i in enumerate(right_node_indices):
        x = boundary.coordinates()[i]
        positional_scale = x[1]/right_max
        x[1] = new_y_right*positional_scale

    # Solve non-linear problem 
    k = 0
    while k < num_nnlin_iter: 
        
        # Assemble momentum matrix and vector 
        Au = assemble(au)
        bu = assemble(Lu)

        # Compute velocity solution 
        [bc.apply(Au, bu) for bc in bcu]
        [bc.apply(u1.vector()) for bc in bcu]
        solve(Au, u1.vector(), bu, "bicgstab", "default")

        # Assemble continuity matrix and vector
        Ap = assemble(ap) 
        bp = assemble(Lp)

        # Compute pressure solution 
        [bc.apply(Ap, bp) for bc in bcp]
        [bc.apply(p1.vector()) for bc in bcp]
        solve(Ap, p1.vector(), bp, "bicgstab", prec)

        # Add time
        if (t > start_sample_time):
          time = np.append(time, t)

        k += 1

    # Save solution to file at every time step
    file_u << u1
    file_p << p1

    if True or t > plot_time:     
        
        s = 'Time t = ' + repr(t) 
        print(s)

        # Plot solution
        plt.figure(figsize=(40, 4))
        plot(u1, title="Velocity")

        plt.figure(figsize=(40, 4))
        plot(p1, title="Pressure")

        for node in surface_nodes:
          print(str(node.position[1]) + " ", end="")
        print()
        
        plot_time += T/plot_freq
        
        plt.show()

    # reshape mesh
    w_new = ALE.move(mesh, boundary)
    w.set_md(w_new)
    uout.h = right_max
    uin.h = left_max
    # this supposedly helps to avoid crashing at solve
    mesh.bounding_box_tree().build(mesh)

    # show reshaped mesh
    print("New Mesh:")
    plt.figure(figsize=(40, 4))
    plot(mesh, title="Mesh")

    # Update time step
    u0.assign(u1)
    t += dt

In [ ]:

# automatically saves the result after simulation,
# this can be done manually if the simulation had to be interrupted. 
!tar -czvf results-NS.tar.gz results-NS
files.download('results-NS.tar.gz')